## An example workflow for D3D

A D3D workflow using local ERAI data.

In [ ]:
#to use the full width of the browser window uncomment the code below and execute the cell
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import pyPoseidon
import pyPoseidon.model as pm
import datetime
from pyPoseidon.utils import data,point
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
pyPoseidon.__version__

In [ ]:
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf
import cartopy.crs as ccrs


hv.notebook_extension('bokeh')
#hv.notebook_extension('matplotlib')

### Define case

In [ ]:
#define in a dictionary the properties of the model..
dic={'minlon':-35., # lat/lon window
     'maxlon':42.,
     'minlat':25.05,
     'maxlat':76.5,
     'solver':'d3d',
     'tag':'eur01',
     'resolution':0.1, #grid resoltuion 
     'step':60, # step for output of map field in d3d
     'rstep': 60*12 ,  # step for restart file
     'start_date':'2017-2-1 0:0:0',
     'end_date':'2017-2-5 0:0:0',
     'ft1':0, # initial forecasting hour per file
     'ft2':11, # final forecasting hour per file
     'meteo':'erai',
     'dem': 'gebco',
     'dpath' : '/Users/brey/DATA/GEBCO_2014_2D.nc',
     'epath':'/Users/brey/DELFT3D/SVN/7545/bin/lnx64/', #folder for solver executables
     'ncores': 4 , #number of cores
     'rpath':'/Users/brey/Downloads/EUR/D3D/ERAI/', #location of calc folder
     'Dt':1.0, # add any additional relevant setup variable,
     'conda_env': 'mpi3'#, # optional conda env for the solver
  #   'update':['grid'] # optional list of components to be updated
    }

#### Local operational ECMWF files

In [ ]:
start_date = pd.to_datetime(dic['start_date']) 
end_date = pd.to_datetime(dic['end_date']) 

In [ ]:
#keep the years
dr = np.arange(start_date.year,end_date.year+1)
dr

In [ ]:
#creating a sequence of folder from which we read the meteo.

PATH='/Users/brey/DATA/ERAI/'#Path to meteo files

meteo_files = [PATH+'eraInterim_{:04d}.grib'.format(x) for x in dr]
meteo_files

#### update dictionary

In [ ]:
dic.update({'mpaths':meteo_files})

## Initialize

In [ ]:
#initialize a model
b = pm.model(**dic)

### set it up

In [ ]:
b.set(**dic) #set it up 

In [ ]:
b.impl.dem.impl.Dataset

#### Optional adjust the wet area based on a coastline shapefile

In [ ]:
b.impl.dem.impl.adjust('/Users/brey/DATA/COASTLINES/naturalearth/coastline/ne_50m_coastline.shp',**{'nc':50,'ncores':4}) 

In [ ]:
b.impl.dem.impl.Dataset

In [ ]:
b.impl.dem.impl.Dataset.fval.plot()# test coverage

## Save to folder for execution 

In [ ]:
#set the run by saving the files
b.output()

In [ ]:
# save model info for further use
b.save()

In [ ]:
# save all matrices for further use
#b.pickle(path=path)

### execute

In [ ]:
#execute
b.run()

In [ ]:
#%matplotlib inline

## Analysis of output 

In [ ]:
otp = data(**dic)#,tag='eur025')# optional tag for data selection ,sa_date='2010-2-1 5:0:0',se_date='2010-2-1 22:0:0') # optionally set range for tide gauge data acquisition 

In [ ]:
otp.impl.Dataset

In [ ]:
otp.impl.dem.bathymetry.plot()#width=600,cmap='jet')

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = plt.subplot(projection=ccrs.PlateCarree());

gp = ax.scatter(otp.impl.x, otp.impl.y, s=2, color='k',  label='grid points', transform=ccrs.PlateCarree());

pp = ax.scatter(otp.impl.xh, otp.impl.yh, s=2, color='b', label='pressure points', transform=ccrs.PlateCarree());

ax.legend()

ax.coastlines('50m'); ax.gridlines(draw_labels=True);

### Visualize

In [ ]:
%%opts Image [width=650 height=450 colorbar=True] (cmap='jet')
otp.impl.hview('S1').to(hv.Image, ['XZ','YZ'], 'S1')#.hist()

In [ ]:
g50 = gv.feature.coastline(plot=dict(scale='50m'), style=dict(linewidth=1.5))

In [ ]:
%%opts Image [colorbar=True width=650 height=450 toolbar="above", tools=['hover']] (cmap='viridis')
#otp.impl.gview('S1').to(gv.Image, ['XZ','YZ'],dynamic=True) * g50 * gf.borders() #Faster
otp.impl.gview('S1').to(gv.Image, ['XZ','YZ']) * g50 * gf.borders()

In [ ]:
otp.impl.frames(['S1'],title='SSH',vmin=-.5, vmax=.5, nv=20)

In [ ]:
otp.impl.frames(['U1Z','V1Z'],title='Vel',scale=.1)

## Observation points / Validation

In [ ]:
otp.impl.obs.locations

In [ ]:
# Get with index number
p = otp.impl.obs.iloc(2)
print p.head()
print p.tail()

In [ ]:
#get with Station Name  
p = otp.impl.obs.loc('Denmark - Hirtshals')
p.head()

In [ ]:
#plot
ax = p.plot()

### sample nearest point from simulation 

In [ ]:
 plat,plon = otp.impl.obs.locations.loc[2,['lat','lon']]

In [ ]:
#ts = otp.vars.S1.sel(XZ=[lon], method='nearest', tolerance=5).sel(YZ=[lat], method='nearest', tolerance=5)
#ts = ts.squeeze().to_pandas()
#ts.head()

In [ ]:
ts = point(lon=plon,lat=plat,data=otp.impl)
ts.tseries(var='S1')

In [ ]:
ts.S1.head()

In [ ]:
ts.S1.plot()

In [ ]:
## Join the graphs
ax = p.plot(figsize=(14,10))
ts.S1.plot(ax=ax,color=['r'])
ax.legend(list(p.columns.values) + ['D3D'], loc='center left',bbox_to_anchor=(1.1, 0.5))

In [ ]:
## Join the graphs
#p.plot(figsize=(14,10)) * ts.S1.plot(label='d3d',color='red')